In [1]:
import mediapipe as mp

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils


In [3]:
import pyautogui

In [4]:
screen_width, screen_height = pyautogui.size()
previous_cursor_x, previous_cursor_y = 0, 0
cursor_smoothening = 7

In [5]:
import cv2

In [6]:
webcam_capture = cv2.VideoCapture(0)